## Data Prep and Modeling

In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer
from tensorflow.keras.layers import InputLayer, Conv2D, MaxPool2D, Flatten, Dense, BatchNormalization, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.metrics import BinaryAccuracy, FalsePositives, FalseNegatives, TruePositives, TrueNegatives, Precision, Recall, AUC
from tensorflow.keras.callbacks import LearningRateScheduler
import sklearn
from sklearn.metrics import confusion_matrix
import seaborn as sns

In [2]:
dataset, dataset_info = tfds.load('malaria', with_info=True, as_supervised=True, split=['train'], shuffle_files=True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Shuffling /root/tensorflow_datasets/malaria/1.0.0.incompleteD5QSLQ/malaria-train.tfrecord*...:   0%|          …

Dataset malaria downloaded and prepared to /root/tensorflow_datasets/malaria/1.0.0. Subsequent calls will reuse this data.


In [3]:
TRAIN_RATIO = 0.6
VAL_RATIO = 0.2
TEST_RATIO = 0.2
DATASET_SIZE = len(dataset[0])

def splits(dataset, TRAIN_RATIO, VAL_RATIO, TEST_RATIO):
  train_dataset = dataset.take(int(TRAIN_RATIO*DATASET_SIZE))

  val_test_dataset = dataset.skip(int(TRAIN_RATIO*DATASET_SIZE))
  val_dataset = val_test_dataset.take(int(VAL_RATIO*DATASET_SIZE))

  test_dataset = val_test_dataset.skip(int(VAL_RATIO*DATASET_SIZE))
  return train_dataset, val_dataset, test_dataset

train_dataset, val_dataset, test_dataset = splits(dataset[0], TRAIN_RATIO, VAL_RATIO, TEST_RATIO)

IM_SIZE = 224
def resize_rescale(image, label):
  return tf.image.resize(image, (IM_SIZE, IM_SIZE))/255.0, label

train_dataset = train_dataset.map(resize_rescale)
val_dataset = val_dataset.map(resize_rescale)
test_dataset = test_dataset.map(resize_rescale)

BATCH_SIZE = 32
train_dataset = train_dataset.shuffle(buffer_size=8, reshuffle_each_iteration=True).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
val_dataset = val_dataset.shuffle(buffer_size=8).batch(BATCH_SIZE)
# test_dataset = test_dataset.shuffle(buffer_size=8).batch(BATCH_SIZE)

In [4]:
model = tf.keras.Sequential([InputLayer(input_shape=(IM_SIZE, IM_SIZE, 3)),

                             Conv2D(filters=6, kernel_size=3, strides=1, padding='valid', activation='relu'),
                             BatchNormalization(),
                             MaxPool2D(pool_size=2, strides=2),

                             Conv2D(filters=16, kernel_size=3, strides=1, padding='valid', activation='relu'),
                             BatchNormalization(),
                             MaxPool2D(pool_size=2, strides=2),

                             Flatten(),

                             Dense(100, activation='sigmoid'),
                             BatchNormalization(),
                             Dense(10, activation='sigmoid'),
                             BatchNormalization(),
                             Dense(1, activation='sigmoid')]) # 0 or 1
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 222, 222, 6)       168       
                                                                 
 batch_normalization (BatchN  (None, 222, 222, 6)      24        
 ormalization)                                                   
                                                                 
 max_pooling2d (MaxPooling2D  (None, 111, 111, 6)      0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 109, 109, 16)      880       
                                                                 
 batch_normalization_1 (Batc  (None, 109, 109, 16)     64        
 hNormalization)                                                 
                                                        

## Model Checkpointing

https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/ModelCheckpoint

In [5]:
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath='checkpoints', # where to save model
    monitor = 'val_loss',
    verbose = 0,
    save_best_only = False,
    save_weights_only = False,
    mode = 'auto',
    save_freq='epoch',
    options=None,
    initial_value_threshold=None
)

metrics = [BinaryAccuracy(name='accuracy'),
           Precision(name='precision'),
           Recall(name='recall'),
           AUC(name='auc')]
model.compile(optimizer=Adam(learning_rate=0.001),
              loss=BinaryCrossentropy(),
              metrics=metrics)
history = model.fit(train_dataset, validation_data=val_dataset, callbacks=[checkpoint_callback], epochs=5, verbose=1)

Epoch 1/5
517/517 [==============================] - ETA: 0s - loss: 0.6209 - accuracy: 0.6603 - precision: 0.6450 - recall: 0.7113 - auc: 0.7169

517/517 [==============================] - 61s 89ms/step - loss: 0.6209 - accuracy: 0.6603 - precision: 0.6450 - recall: 0.7113 - auc: 0.7169 - val_loss: 0.6257 - val_accuracy: 0.6757 - val_precision: 0.6452 - val_recall: 0.7847 - val_auc: 0.7173
Epoch 2/5
516/517 [============================>.] - ETA: 0s - loss: 0.4782 - accuracy: 0.7812 - precision: 0.7604 - recall: 0.8206 - auc: 0.8539

517/517 [==============================] - 45s 86ms/step - loss: 0.4781 - accuracy: 0.7812 - precision: 0.7604 - recall: 0.8207 - auc: 0.8540 - val_loss: 0.4606 - val_accuracy: 0.8055 - val_precision: 0.7412 - val_recall: 0.9407 - val_auc: 0.8764
Epoch 3/5
516/517 [============================>.] - ETA: 0s - loss: 0.3750 - accuracy: 0.8497 - precision: 0.8309 - recall: 0.8778 - auc: 0.9110

517/517 [==============================] - 42s 81ms/step - loss: 0.3750 - accuracy: 0.8497 - precision: 0.8310 - recall: 0.8776 - auc: 0.9110 - val_loss: 0.5520 - val_accuracy: 0.7093 - val_precision: 0.7637 - val_recall: 0.6076 - val_auc: 0.8116
Epoch 4/5
516/517 [============================>.] - ETA: 0s - loss: 0.2934 - accuracy: 0.8918 - precision: 0.8774 - recall: 0.9107 - auc: 0.9432

517/517 [==============================] - 42s 82ms/step - loss: 0.2933 - accuracy: 0.8918 - precision: 0.8774 - recall: 0.9107 - auc: 0.9433 - val_loss: 0.2903 - val_accuracy: 0.8922 - val_precision: 0.8496 - val_recall: 0.9541 - val_auc: 0.9493
Epoch 5/5
517/517 [==============================] - ETA: 0s - loss: 0.2786 - accuracy: 0.8963 - precision: 0.8830 - recall: 0.9134 - auc: 0.9487

517/517 [==============================] - 42s 82ms/step - loss: 0.2786 - accuracy: 0.8963 - precision: 0.8830 - recall: 0.9134 - auc: 0.9487 - val_loss: 0.4955 - val_accuracy: 0.8269 - val_precision: 0.7541 - val_recall: 0.9718 - val_auc: 0.9298


In [6]:
!ls checkpoints

assets	fingerprint.pb	keras_metadata.pb  saved_model.pb  variables
